In [1]:
import os, pickle
import numpy as np
import pandas as pd

import gseapy as gp

ONTO_PATH = "/data2/zhoujb/project/cowpea_project/rawData/geneOntology/"
GOKEGG_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/GOKEGG"

In [2]:
go_df = pd.read_table(os.path.join(ONTO_PATH, "go_gene_id.txt"))
go_df["Term_list"] = go_df["Term"].str.split(",")

cc_df = go_df[go_df["GO_TYPE"]=="C"].copy()
cc_d = cc_df[["GO_NAME", "Term_list"]].set_index("GO_NAME")["Term_list"].to_dict()

mf_df = go_df[go_df["GO_TYPE"]=="F"].copy()
mf_d = mf_df[["GO_NAME", "Term_list"]].set_index("GO_NAME")["Term_list"].to_dict()

bp_df = go_df[go_df["GO_TYPE"]=="P"].copy()
bp_d = bp_df[["GO_NAME", "Term_list"]].set_index("GO_NAME")["Term_list"].to_dict()

In [3]:
data_term = pd.read_table(os.path.join(GOKEGG_PATH, "traits_region_gene.txt"))

cc_gsea_res = pd.DataFrame()
for group in data_term.groupby("traits"):
    group_name, group_data = group
    gene_list = list(set([str(x) for x in group_data['symbol'].values]))

    enr2 = gp.enrich(gene_list=gene_list,
                 gene_sets=cc_d, # kegg is a dict object
                 background=32563, # or "hsapiens_gene_ensembl", or int, or text file, or a list of genes
                 outdir=None,
                 verbose=True)

    tmp_res = enr2.results
    tmp_res = tmp_res[tmp_res["Adjusted P-value"]<=0.05].copy()
    tmp_res["traits"] = group_name

    cc_gsea_res = pd.concat([cc_gsea_res, tmp_res], axis=0, ignore_index=True)
    cc_gsea_res["GO_type"] = "CC"

mf_gsea_res = pd.DataFrame()
for group in data_term.groupby("traits"):
    group_name, group_data = group
    gene_list = list(set([str(x) for x in group_data['symbol'].values]))

    enr2 = gp.enrich(gene_list=gene_list,
                 gene_sets=mf_d, # kegg is a dict object
                 background=32563, # or "hsapiens_gene_ensembl", or int, or text file, or a list of genes
                 outdir=None,
                 verbose=True)

    tmp_res = enr2.results
    tmp_res = tmp_res[tmp_res["Adjusted P-value"]<=0.05].copy()
    tmp_res["traits"] = group_name

    mf_gsea_res = pd.concat([mf_gsea_res, tmp_res], axis=0, ignore_index=True)
    mf_gsea_res["GO_type"] = "MF"

bp_gsea_res = pd.DataFrame()
for group in data_term.groupby("traits"):
    group_name, group_data = group
    gene_list = list(set([str(x) for x in group_data['symbol'].values]))

    enr2 = gp.enrich(gene_list=gene_list,
                 gene_sets=bp_d, # kegg is a dict object
                 background=32563, # or "hsapiens_gene_ensembl", or int, or text file, or a list of genes
                 outdir=None,
                 verbose=True)

    tmp_res = enr2.results
    tmp_res = tmp_res[tmp_res["Adjusted P-value"]<=0.05].copy()
    tmp_res["traits"] = group_name

    bp_gsea_res = pd.concat([bp_gsea_res, tmp_res], axis=0, ignore_index=True)
    bp_gsea_res["GO_type"] = "BP"

data_con = pd.concat([cc_gsea_res, mf_gsea_res, bp_gsea_res], axis=0, ignore_index=True)

print("ALL DONE")

2024-07-19 16:26:40,029 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,031 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,092 [INFO] Done.
2024-07-19 16:26:40,100 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,102 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,180 [INFO] Done.
2024-07-19 16:26:40,189 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,191 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,294 [INFO] Done.
2024-07-19 16:26:40,317 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,321 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,386 [INFO] Done.
2024-07-19 16:26:40,393 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,395 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,430 [INFO] Done.
2024-07-19 16:26:40,436 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40,438 [INFO] Run: gs_ind_0 
2024-07-19 16:26:40,482 [INFO] Done.
2024-07-19 16:26:40,493 [INFO] Input dict object named with gs_ind_0
2024-07-19 16:26:40

ALL DONE


In [4]:
data_con

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score,Genes,traits,GO_type
0,gs_ind_0,extracellular region,14/184,7.564617e-06,8.169786e-04,4.683310,55.225725,LOC114179842;LOC114194040;LOC114194200;LOC1141...,GNP,CC
1,gs_ind_0,extracellular space,8/77,7.869384e-05,4.249467e-03,6.687004,63.191826,LOC114179842;LOC114162120;LOC114179029;LOC1141...,GNP,CC
2,gs_ind_0,phosphopantothenoylcysteine decarboxylase complex,2/3,9.746666e-04,3.508800e-02,90.391575,626.722307,LOC114185898;LOC114183207,GNP,CC
3,gs_ind_0,Cul4A-RING E3 ubiquitin ligase complex,8/20,6.175768e-10,5.928737e-08,40.491944,858.640490,LOC114179456;LOC114178099;LOC114178098;LOC1141...,PL,CC
4,gs_ind_0,exocyst,7/50,1.896326e-05,9.102364e-04,10.237729,111.314905,LOC114195564;LOC114195074;LOC114195075;LOC1141...,PL,CC
...,...,...,...,...,...,...,...,...,...,...
134,gs_ind_0,regulation of COPII vesicle coating,2/6,1.834225e-03,2.367817e-02,49.067977,309.183870,LOC114179478;LOC114179435,TSW,BP
135,gs_ind_0,regulation of protein serine/threonine phospha...,7/38,2.001246e-07,9.472562e-06,21.303729,328.595654,LOC114177279;LOC114177113;LOC114177169;LOC1141...,TSW,BP
136,gs_ind_0,triterpenoid biosynthetic process,7/16,2.244183e-10,3.186740e-08,70.686992,1570.488917,LOC114188179;LOC114187122;LOC114187127;LOC1141...,TSW,BP
137,gs_ind_0,vesicle organization,2/4,7.447237e-04,1.510725e-02,88.327846,636.181079,LOC114179478;LOC114179435,TSW,BP


In [5]:
with open(os.path.join(GOKEGG_PATH, "go_gsea_res.pkl"), "wb") as f:
    pickle.dump(data_con, f)